In [1]:
import os
import sys

import timm
import torchvision.transforms as T
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader, random_split

import hydra
from pytorch_lightning import LightningModule, Trainer
from dotenv import load_dotenv, dotenv_values

sys.argv = ['']
# 환경변수 읽기
if (python_path := dotenv_values().get('PYTHONPATH')) and python_path not in sys.path: sys.path.append(python_path)

from src.models.CNN import SimpleCNN, ResNet
from src.utils import config

@hydra.main(config_path=config.CONFIGS_DIR, config_name="config", version_base=None)
def main(cfg):
    print(cfg)

    transform = T.Compose([T.ToTensor()])
    dataset = CIFAR10(root=config.DATA_DIR, train=True, download=True, transform=transform)

    train_size = int(len(dataset) * (1 - cfg.data.valid_split))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

    train_loader = DataLoader(train_dataset, batch_size=cfg.data.batch_size, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=cfg.data.batch_size, shuffle=False, num_workers=2)

    if 'resnet' in cfg.model.model.model_name:
        model = ResNet(cfg)
    else:
        model = SimpleCNN(cfg)

    trainer = Trainer(default_root_dir=config.OUTPUTS_DIR, max_epochs=cfg.trainer.max_epochs, accelerator=cfg.trainer.accelerator)
    trainer.fit(model, train_loader, val_loader)


if __name__ == "__main__":
    main()

/data/ephemeral/home/mywork/python_work/git/gx-train/.venv/lib/python3.10/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'config': Defaults list is missing `_self_`. See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)


{'data': {'data_dir': './CIFAR10_DATASET', 'batch_size': 64, 'valid_split': 0.2}, 'model': {'model': {'_target_': 'timm.create_model', 'model_name': 'resnet18', 'num_classes': 10, 'pretrained': True}}, 'optimizer': {'_target_': 'torch.optim.Adam', 'lr': 0.001, 'weight_decay': 0.0001}, 'scheduler': {'_target_': 'torch.optim.lr_scheduler.StepLR', 'step_size': 10, 'gamma': 0.5}, 'logger': {'project': 'lightning_with_hydra', 'name': 'test'}, 'callback': {'monitor': 'valid_loss', 'mode': 'min', 'patience': 3, 'logging_interval': 'epoch'}, 'trainer': {'max_epochs': 10, 'accelerator': 'auto'}}


100%|██████████| 170M/170M [00:12<00:00, 13.6MB/s] 


[2025-06-23 07:29:07,127][timm.models._builder][INFO] - Loading pretrained weights from Hugging Face hub (timm/resnet18.a1_in1k)
[2025-06-23 07:29:07,444][timm.models._hub][INFO] - [timm/resnet18.a1_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.
[2025-06-23 07:29:07,467][timm.models._builder][INFO] - Missing keys (fc.weight, fc.bias) discovered while loading pretrained weights. This is expected if model is being adapted.


💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/data/ephemeral/home/mywork/python_work/git/gx-train/.venv/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
/data/ephemeral/home/mywork/python_work/git/gx-train/.venv/lib/python3.10/site-packages/pytorch_lightning/t

Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.
